##Set Up

In [ ]:
!pip install --upgrade google-cloud-videointelligence
!pip install google-cloud-speech

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/"Systems '22"/

/content/gdrive/.shortcut-targets-by-id/1ohwBGdC8wri16R_SjnGnOM_B619TDepK/Systems '22


In [ ]:
import sys
from google.colab import auth
if "google.colab" in sys.modules:
  auth.authenticate_user()

In [ ]:
BILLING_ACCOUNT_ID = '01074B-EAE851-71F16B'

GCP_PROJECT_ID = 'stoked-dominion-341317'
PROJECT_NUMBER = '725342657078'

!gcloud beta billing projects link $GCP_PROJECT_ID --billing-account $BILLING_ACCOUNT_ID

In [ ]:
!gcloud beta billing accounts list

In [ ]:
!gcloud services --project $GCP_PROJECT_ID enable ml.googleapis.com cloudbuild.googleapis.com

Operation "operations/acat.p2-725342657078-8f2534ff-ae89-4e1e-ad17-5b3cdc5295ae" finished successfully.


In [ ]:
SERVICE_ACCOUNT_NAME ='ytnotebook'

SERVICE_ACCOUNT_EMAIL = f'{SERVICE_ACCOUNT_NAME}@{GCP_PROJECT_ID}.iam.gserviceaccount.com'
#!gcloud iam --project $GCP_PROJECT_ID service-accounts create $SERVICE_ACCOUNT_NAME
!gcloud projects add-iam-policy-binding $GCP_PROJECT_ID \
    --member serviceAccount:$SERVICE_ACCOUNT_EMAIL \
    --role=roles/editor

In [ ]:
DEFAULT_AI_PLATFORM_SERVICE_ACCOUNT = f'service-{PROJECT_NUMBER}@cloud-ml.google.com.iam.gserviceaccount.com'

!gcloud iam --project $GCP_PROJECT_ID service-accounts add-iam-policy-binding \
--role=roles/iam.serviceAccountAdmin \
--member=serviceAccount:$DEFAULT_AI_PLATFORM_SERVICE_ACCOUNT \
$SERVICE_ACCOUNT_EMAIL

In [ ]:
from google.cloud import storage
import os
import pprint

"""
this is the path to the API credentials. Modify this to the path of your own GCP credential.
"""
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'Scripts/stoked-dominion-341317-24484c3fbb6f.json'

## Upload audios to GCP

In [ ]:
import io
import argparse
from google.cloud import storage
from google.cloud import speech
import json

In [ ]:
# assign the audios folder here, we will upload audios to GCP from here
AUDIO_FOLDER = "text/audios-wav/"
UP_LOAD_FILE_ROUTE = AUDIO_FOLDER

# assign which bucket the audios should be uploaded to
BUCKET_NAME ="audio_wav_list_capstoneyt_2022"


In [ ]:

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    #Uploads a file to the bucket.

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )


In [ ]:

def bucket_metadata(bucket_name):
    #Prints out a bucket's metadata.
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)

    print("ID: {}".format(bucket.id))
    print("Name: {}".format(bucket.name))compared
    print("Storage Class: {}".format(bucket.storage_class))
    print("Location: {}".format(bucket.location))
    print("Location Type: {}".format(bucket.location_type))
    print("Cors: {}".format(bucket.cors))
    print(
        "Default Event Based Hold: {}".format(bucket.default_event_based_hold)
    )
    print("Default KMS Key Name: {}".format(bucket.default_kms_key_name))
    print("Metageneration: {}".format(bucket.metageneration))
    print(
        "Retention Effective Time: {}".format(
            bucket.retention_policy_effective_time
        )
    )
    print("Retention Period: {}".format(bucket.retention_period))
    print("Retention Policy Locked: {}".format(bucket.retention_policy_locked))
    print("Requester Pays: {}".format(bucket.requester_pays))
    print("Self Link: {}".format(bucket.self_link))
    print("Time Created: {}".format(bucket.time_created))
    print("Versioning Enabled: {}".format(bucket.versioning_enabled))
    print("Labels:")
    print.pprint(bucket.labels)

def list_blobs_with_prefix(bucket_name, prefix, out_list, delimiter=None):
    storage_client = storage.Client()
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=delimiter)

    #print("Blobs:")
    for blob in blobs:
      #print(blob.name)
      out_list.append(blob.name)

def upload_files(i,j,filenames):
    #file_names = os.listdir(UP_LOAD_FILE_ROUTE)
    #print("remaining: ", len(file_names) - len(outputAudio))
    for file1 in file_names[i:j]:
        if AUDIO_FOLDER[5:]+file1 not in outputAudio:
          storage_client = storage.Client()
          bucket= storage_client.bucket(BUCKET_NAME)
          source_file_route = os.path.join(UP_LOAD_FILE_ROUTE, file1)
          blob = bucket.blob(AUDIO_FOLDER[5:]+file1)
          blob.upload_from_filename(source_file_route)
          print(
              "File {} uploaded to {}.".format(
                  source_file_route, file1
              )
          )
          bucket=None



In [ ]:
outputAudio=[]
list_blobs_with_prefix(bucket_name=BUCKET_NAME, prefix="audios-wav/", delimiter="/",out_list=outputAudio)


file_names = list(set(os.listdir(UP_LOAD_FILE_ROUTE))-set([x[11:] for x in outputAudio]))
print("Remaining audios number: ", len(file_names) - len(outputAudio))


MIN=0
MAX=len(file_names)

In [ ]:
"""
re-enter the upload_files method after every 4 uploads
"""
while len(file_names)>4:
  outputAudio=[]
  list_blobs_with_prefix(bucket_name="audio_wav_list_capstoneyt_2022", prefix="audios-wav/", delimiter="/",out_list=outputAudio)
  file_names = list(set(os.listdir(UP_LOAD_FILE_ROUTE))-set([x[11:] for x in outputAudio]))
  upload_files(0,4,file_names)
  #from google.colab import drive
  #drive.mount("/content/gdrive", force_remount=True)
upload_files(0,len(file_names),file_names)

## Speech to text

In [ ]:
# assign SAVE PATH for recognized text
AUDIO_PATH = "./text/audios-wav"
SAVE_PATH = "./text/transcript_text/unlabeled/raw_text/"
AUDIO_ID_LIST = os.listdir(AUDIO_PATH)
AUDIO_ID_LIST

In [ ]:
def get_processed_filename_set():
  """
  Get the set of processed filenames.
  """
    processed_filename_set = os.listdir(SAVE_PATH)
    processed_filename_set = [i.split('.')[0] for i in processed_filename_set]
    return processed_filename_set

In [ ]:
def sample_long_running_recognize(storage_uri, filename):
    """
    Transcribe long audio file from Cloud Storage using asynchronous speech
    recognition
    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech.SpeechClient()
    print("connect client")

    # storage_uri = 'gs://cloud-samples-data/speech/brooklyn_bridge.raw'

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 16000

    # The language of the supplied audio
    language_code = "en-US"

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = speech.RecognitionConfig.AudioEncoding.LINEAR16
    config = {
        "audio_channel_count": 2,
        "enable_separate_recognition_per_channel": False,
        # "sample_rate_hertz": sample_rate_hertz,
        "language_code": language_code,
        "enable_automatic_punctuation": True,
        "encoding": encoding,
    }
    audio = {"uri": storage_uri}
    operation = client.long_running_recognize(request={"config": config, "audio": audio})

    print(u"Waiting for operation to complete...")
    response = operation.result()

    filename = filename.split('.')[0].split('/')[-1]

    transcript_json = []
    transcript_str = ''
    for idx, result in enumerate(response.results):
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))
        print(u"Confidence: {}".format(alternative.confidence))
        transcript_str += alternative.transcript
        transcript_json.append({"transcript":alternative.transcript, "confidence":alternative.confidence})
    with open(os.path.join(SAVE_PATH, filename+'.txt'), 'w') as f1:
        f1.write(transcript_str)
    with open(os.path.join(SAVE_PATH, filename+'.json'), 'w') as f2:
        json.dump(transcript_json, f2)

In [ ]:
def get_exist_filename_set(bucket_name):

    client = storage.Client()
    # bucket = storage.Bucket("fyanvideos")
    blob_list = client.list_blobs(bucket_name)
    filenames = set()
    for blob in blob_list:
        filenames.add(blob.name)
        # print(blob.name)
    return filenames

In [ ]:
def main():
  """
  Start recognizing...
  """
    filenames = get_exist_filename_set(BUCKET_NAME)
    processed_filename_set = get_processed_filename_set()
    processed=0
    
    for filename in list(filenames)[800:]:
        pre_name = filename.split('.')[0].split('/')[-1]
        if pre_name not in processed_filename_set:
            if processed>0:
              print('already processed %d'%processed)
              processed=0
            storage_uri = 'gs://{}/{}'.format(BUCKET_NAME, filename)
            sample_long_running_recognize(storage_uri, filename)  
        else:
            processed+=1

In [ ]:
main()